# Sistema de Busca em Background - SmartQuote

## Resolução de Problemas de Comunicação Worker-Parent

Este notebook documenta a implementação e correção do sistema de busca em background que resolve o problema de bloqueio do servidor durante pesquisas longas (120+ segundos).

### Problema Original
- Requests de busca bloqueavam o servidor por 2+ minutos
- Necessidade de executar buscas em paralelo sem bloquear o thread principal
- Implementação de arquitetura com processes filhos para background jobs

## 1. Setting Up the Environment

### Módulos Node.js para Worker Management

O sistema utiliza os seguintes módulos principais:

In [ ]:
// JobManager.ts - Gerenciamento de processes filhos
import { fork, ChildProcess, spawn } from 'child_process';
import { v4 as uuidv4 } from 'uuid';
import path from 'path';

// Worker Process - buscaWorker.ts
// Executa a busca em processo separado
// Comunica progresso via stdout/stderr

// Controller - BuscaController.ts
// Endpoints para:
// - POST /background - Inicia job e retorna imediatamente
// - GET /job/:id - Status do job
// - GET /jobs - Lista todos os jobs

## 2. Identifying JSON Parsing Errors

### Problemas Identificados

Os erros de parsing JSON ocorreram devido a:

1. **Mistura de logs e JSON**: Worker enviava logs + mensagens JSON no mesmo stream
2. **Caracteres especiais**: Emojis e caracteres unicode corrompiam o JSON
3. **Mensagens do dotenv**: Logs automáticos do dotenv misturados com JSON

In [ ]:
// ERRO ORIGINAL - Mixed output
// stdout continha:
// "[dotenv@17.2.1] injecting env..."
// "🔧 Worker iniciado..."
// "{\"progresso\": {\"etapa\": \"busca\"}}"

// Resultado: JSON.parse() falhava nos logs não-JSON

// SOLUÇÃO - Separação de streams
function enviarMensagem(message) {
  // Prefixo especial para identificar JSON válido
  console.log('WORKER_MSG:' + JSON.stringify(message));
}

function log(message) {
  // Logs via stderr para não interferir
  console.error(`[WORKER] ${message}`);
}

## 3. Analyzing Worker Communication Issues

### Arquitetura Original vs Corrigida

**Problema**: `fork()` com TypeScript em desenvolvimento não funcionava corretamente

**Solução**: `spawn()` com `ts-node` para execução direta de TypeScript

In [ ]:
// JobManager.ts - Solução final
private executarJob(jobId: string): void {
  const workerPath = path.join(__dirname, '../workers/buscaWorker.ts');
  
  // Usar spawn com ts-node para executar TypeScript diretamente
  const childProcess = spawn('npx', ['ts-node', workerPath], {
    stdio: ['pipe', 'pipe', 'pipe', 'ipc'],
    cwd: process.cwd()
  });
  
  // Enviar job data via stdin
  childProcess.stdin?.write(JSON.stringify(jobData) + '\n');
  
  // Escutar apenas mensagens com prefixo WORKER_MSG:
  childProcess.stdout?.on('data', (data) => {
    const lines = data.toString().trim().split('\n');
    lines.forEach((line) => {
      if (line.startsWith('WORKER_MSG:')) {
        const jsonStr = line.substring('WORKER_MSG:'.length);
        const message = JSON.parse(jsonStr);
        this.processarMensagemDoFilho(jobId, message);
      }
    });
  });
}

## 4. Implementing Message Validation

### Sistema de Prefixos para Validação

Implementação de protocolo simples para garantir que apenas JSON válido seja processado:

In [ ]:
// buscaWorker.ts - Envio de mensagens
interface ProgressMessage {
  progresso: {
    etapa: 'busca' | 'salvamento';
    fornecedores?: number;
    produtos?: number;
    detalhes?: string;
  };
}

interface ResultMessage {
  status: 'sucesso' | 'erro';
  produtos?: any[];
  salvamento?: {
    salvos: number;
    erros: number;
    detalhes: any[];
  };
  tempoExecucao?: number;
  erro?: string;
}

// Validação no envio
function enviarMensagem(message: ProgressMessage | ResultMessage) {
  // Validar estrutura antes de enviar
  if (!message || typeof message !== 'object') {
    log('Erro: Mensagem inválida');
    return;
  }
  
  console.log('WORKER_MSG:' + JSON.stringify(message));
}

## 5. Creating Error Handling Solutions

### Correção do Erro de Database (Preço NULL)

Outro problema identificado foi produtos sem preço causando constraint violations:

In [ ]:
// ProdutoService.ts - Correção da conversão de preço
private converterPrecoParaCentavos(precoString: string | null | undefined): number {
  try {
    // Se preço é null, undefined ou string vazia, retorna 0
    if (!precoString || precoString.trim() === '') {
      console.warn('Preço vazio ou nulo, usando 0');
      return 0;
    }
    
    // Remove símbolos de moeda e espaços
    const numeroLimpo = precoString.replace(/[^\d.,]/g, '');
    
    if (!numeroLimpo) {
      console.warn('Preço sem números válidos:', precoString);
      return 0;
    }
    
    // Processamento de formato brasileiro vs americano
    // ... lógica de conversão ...
    
    // Validar se é um número válido
    if (isNaN(resultado)) {
      console.warn('Resultado de conversão inválido:', precoString);
      return 0;
    }
    
    return resultado;
  } catch (error) {
    console.warn('Erro ao converter preço:', precoString, error);
    return 0; // preço padrão em caso de erro
  }
}

## 6. Testing Message Formatting

### Teste End-to-End do Sistema

Script de teste criado para validar todo o fluxo:

In [ ]:
// teste-background.js - Teste completo
async function testarBuscaBackground() {
  const baseUrl = 'http://localhost:2000/api/busca-automatica';
  
  // 1. Iniciar busca em background
  const resposta = await fetch(`${baseUrl}/background`, {
    method: 'POST',
    headers: { 'Content-Type': 'application/json' },
    body: JSON.stringify({ produto: 'notebook' })
  });
  
  const dadosBusca = await resposta.json();
  const jobId = dadosBusca.jobId;
  
  // 2. Monitorar progresso
  while (tentativas < maxTentativas) {
    const respostaStatus = await fetch(`${baseUrl}/job/${jobId}`);
    const job = (await respostaStatus.json()).job;
    
    console.log(`Status: ${job.status}`);
    if (job.progresso) {
      console.log(`Etapa: ${job.progresso.etapa}`);
      console.log(`Detalhes: ${job.progresso.detalhes}`);
    }
    
    if (job.status === 'concluido' || job.status === 'erro') {
      break;
    }
    
    await new Promise(resolve => setTimeout(resolve, 10000));
  }
}

## 7. Monitoring and Logging Best Practices

### Sistema de Logs Implementado

**Separação Clara de Responsabilidades:**

1. **stdout**: Apenas mensagens JSON válidas com prefixo `WORKER_MSG:`
2. **stderr**: Logs do worker para debugging
3. **console.log no main**: Logs do JobManager

### Resultados Finais

✅ **Problema resolvido**: Requests agora retornam imediatamente (202 Accepted)

✅ **Background processing**: Jobs executam em paralelo sem bloquear servidor

✅ **Monitoramento**: Sistema de tracking de progresso em tempo real

✅ **Error handling**: Tratamento robusto de erros de comunicação e database

### Performance

- **Antes**: 120+ segundos bloqueando o servidor
- **Depois**: Resposta imediata + processamento paralelo
- **Throughput**: Múltiplas buscas simultâneas possíveis
- **Monitoramento**: Progress tracking granular por etapa

## Endpoints Disponíveis

### Background Search API

```bash
# Iniciar busca em background (retorna imediatamente)
POST /api/busca-automatica/background
{
  "produto": "notebook"
}

# Verificar status de um job
GET /api/busca-automatica/job/{jobId}

# Listar todos os jobs
GET /api/busca-automatica/jobs

# Cancelar job
DELETE /api/busca-automatica/job/{jobId}

# Busca síncrona (método original - ainda disponível)
POST /api/busca-automatica/
```

### Response Examples

**Background Job Iniciado:**
```json
{
  "success": true,
  "message": "Busca iniciada em background",
  "jobId": "62205208-157e-40d4-9e75-39c47a5ff4f6",
  "statusUrl": "/api/busca/job/{jobId}",
  "parametros": {
    "termo": "notebook",
    "numResultados": 3,
    "fornecedores": 2
  }
}
```

**Job Status:**
```json
{
  "success": true,
  "job": {
    "id": "62205208-157e-40d4-9e75-39c47a5ff4f6",
    "status": "executando",
    "progresso": {
      "etapa": "busca",
      "fornecedores": 2,
      "detalhes": "Iniciando busca em 2 fornecedores..."
    }
  }
}
```